In [1]:
import torch
import commons
import utils
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence
from scipy.io.wavfile import write
import IPython.display as ipd

def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

CONFIG_PATH = "./configs/vits2_ljs_ring.json"
hps = utils.get_hparams_from_file(CONFIG_PATH)

MODEL_PATH = "./logs/rc/G.pth"
    
if (
    "use_mel_posterior_encoder" in hps.model.keys()
    and hps.model.use_mel_posterior_encoder == True
):
    print("Using mel posterior encoder for VITS2")
    posterior_channels = 80  # vits2
    hps.data.use_mel_posterior_encoder = True
else:
    print("Using lin posterior encoder for VITS1")
    posterior_channels = hps.data.filter_length // 2 + 1
    hps.data.use_mel_posterior_encoder = False

net_g = SynthesizerTrn(
    len(symbols),
    posterior_channels,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model
).cuda()
_ = net_g.eval()
_ = utils.load_checkpoint(MODEL_PATH, net_g, None)

DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=1039)
           2	LOAD_FAST(arg=0, lineno=1042)
           4	LOAD_CONST(arg=1, lineno=1042)
           6	BINARY_SUBSCR(arg=None, lineno=1042)
           8	LOAD_FAST(arg=0, lineno=1042)
          10	LOAD_CONST(arg=2, lineno=1042)
          12	BINARY_SUBSCR(arg=None, lineno=1042)
          14	COMPARE_OP(arg=4, lineno=1042)
          16	LOAD_FAST(arg=0, lineno=1042)
          18	LOAD_CONST(arg=1, lineno=1042)
          20	BINARY_SUBSCR(arg=None, lineno=1042)
          22	LOAD_FAST(arg=0, lineno=1042)
          24	LOAD_CONST(arg=3, lineno=1042)
          26	BINARY_SUBSCR(arg=None, lineno=1042)
          28	COMPARE_OP(arg=5, lineno=1042)
          30	BINARY_AND(arg=None, lineno=1042)
          32	RETURN_VALUE(arg=None, lineno=1042)
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:state.pc_initial: State(pc_initial=0 nstack_

/root/anaconda3/envs/ring_conformer/lib/python3.10/site-packages/ring_attention_pytorch/ring_attention.py:128: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/root/anaconda3/envs/ring_conformer/lib/python3.10/site-packages/ring_attention_pytorch/ring_attention.py:164: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/root/anaconda3/envs/ring_conformer/lib/python3.10/site-packages/ring_attention_pytorch/ring_flash_attention_cuda.py:355: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)


Using mel posterior encoder for VITS2


/root/anaconda3/envs/ring_conformer/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/data/RC_VITS/utils.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` fo

INFO:root:Loaded checkpoint './logs/rc/G.pth' (iteration 920)


In [3]:
import time
import numpy as np

num_iterations = 1
inference_times = []
rtf_values = []
TEXTS = ["The poorer prisoners were not"]

for idx, text in enumerate(TEXTS):
    stn_tst = get_text(text, hps)
    with torch.no_grad():
        x_tst = stn_tst.cuda().unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()

        for _ in range(num_iterations):
            start = time.perf_counter()
            audio = (
                net_g.infer(
                    x_tst, x_tst_lengths, noise_scale=0.667, noise_scale_w=0.8, length_scale=1
                )[0][0, 0]
                .data.cpu()
                .float()
                .numpy()
            )
            
            end = time.perf_counter()
            inference_time = end - start
            inference_times.append(inference_time)
            
            audio_length = len(audio) / hps.data.sampling_rate
            rtf = inference_time / audio_length
            rtf_values.append(rtf)
            
            ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=True))
            # write(data=audio, rate=hps.data.sampling_rate, filename=f'results.wav')

torch.Size([1, 66, 2305])
torch.Size([1, 33, 2305]) torch.Size([1, 33, 2305])
